# Lab 20: Persistent Memory with Redis - Resuming Conversations

This lab demonstrates how to resume and continue conversations using Redis-based persistent memory. You'll learn:
- How persistent memory survives application restarts
- Resuming conversations from previous sessions stored in Redis
- Understanding conversation continuity across different execution contexts
- Leveraging Redis for stateful conversational applications
- Comparing with traditional in-memory approaches that lose state

In [ ]:
# Re-initialize the Redis-based conversation system
# This simulates starting a new application instance or notebook session
# The setup is identical to Lab 19 but demonstrates persistence across sessions
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

import os
# API key configuration (same as before)
os.environ["OPENAI_API_KEY"] = "your-api-key"

# Recreate the model and prompt template
# Even though we're "restarting", Redis preserves conversation history
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),  # Will load from Redis
        ("human", "{input}"),
    ]
)
base_chain = prompt | model

In [ ]:
# Same Redis configuration as Lab 19
# The connection points to the same Redis instance with preserved data
REDIS_URL = "redis://localhost:6379/0"

# Store variable (unused but maintained for consistency)
store = {}

In [ ]:
# Recreate the Redis-backed conversation system
# This function will connect to the same Redis instance and retrieve existing conversations
def get_message_history(session_id: str) -> RedisChatMessageHistory:
    """Connect to existing or new Redis message history for session"""
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

# Rebuild the chain with Redis memory integration
# This chain can access conversations started in Lab 19
redis_chain = RunnableWithMessageHistory(
    base_chain,                          # Recreated base chain
    get_message_history,                 # Same Redis connection function
    input_messages_key="input",          
    history_messages_key="history",      
)

In [ ]:
# Resume the math conversation from Lab 19 using the same session ID
# Redis has preserved the entire conversation history about cosine
# "Tell me more!" will continue from where Lab 19 left off
# This demonstrates true persistence across application sessions
redis_chain.invoke(
    {"ability": "math", "input": "Tell me more!"},
    config={"configurable": {"session_id": "math-thread1"}},  # Continues from Lab 19
)

In [ ]:
# Resume the physics conversation from Lab 19 using the same session ID
# Redis has maintained the separate physics thread about relativity theory  
# This shows how multiple conversation threads persist independently
# The AI will continue discussing relativity, not cosine
redis_chain.invoke(
    {"ability": "physics", "input": "Tell me more!"},
    config={"configurable": {"session_id": "phy-thread1"}},  # Continues physics from Lab 19
)